In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
class Vertice:
    def __init__(self, dado, direcionado=True):
        self.__dado = dado
        self.__direcionado = direcionado
        self.__arestas = set()
    
    def getDado(self):
        return self.__dado
    
    def setDado(self, dado):
        self.__dado = dado
        
    def getArestas(self):
        return self.__arestas
    
    def adicionarAresta(self, aresta):
        self.__arestas.add(aresta)
        
    def getArestasSaida(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasDeSaida = []
        for aresta in self.__arestas:
            if aresta.getvOrigem() == self:
                arestasDeSaida.append(aresta)
        return arestasDeSaida
    
    def getArestasEntrada(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasSaida = []
        for aresta in self.__arestas:
            if aresta.getvDestino() == self:
                arestasSaida.append(aresta)
        return arestasSaida
    
    def getGrau(self):
        return len(self.getArestasSaida()) + len(self.getArestasEntrada())
    '''
    def getGrauE(self):
        return len(self.getArestasEntrada())

    def getGrauS(self):
        return len(self.getArestasSaida()) 
    '''
    def getAdjacentes(self, v):
        listaVerticesAdjacentes = []
        for arestas_de_saida in v.getArestasSaida():
            listaVerticesAdjacentes.append(arestas_de_saida.getvDestino())
        return listaVerticesAdjacentes

In [2]:
class Aresta:
    def __init__(self, vOrigem, vDestino, valor, direcionada=True):
        self.__vOrigem = vOrigem
        self.__vDestino = vDestino
        self.__valor = valor
        self.__direcionada = direcionada
        self.__vOrigem.adicionarAresta(self)
        self.__vDestino.adicionarAresta(self)
        
        
    def getvOrigem(self):
        return self.__vOrigem
    def getvDestino(self):
        return self.__vDestino
    def getDado(self):
        return self.__valor

In [23]:
from collections import deque
class Grafo:
    def __init__(self, direcionado=True):
        self.__vertices = set()
        self.__arestas  = set()
        self.__direcionado = direcionado
        
    def setVertices(self, vertices):
        self.__vertices = vertices
        
    def setArestas(self, arestas):
        self.__arestas = arestas
        
    def getVertices(self):
        return self.__vertices
    
    def getArestas(self):
        return self.__arestas
    
    #def getHul(self):
        #for v in      
    #def getVertice(self):
        #return len(self.getAresta(getDado))

    def getArestaDado(self, dado):
        for v in self.__vertices:
            if v.getDado() == dado:
                return v
        return None
    
#-------11/05-------#
    def adicionarVertice(self, dado):
        if self.buscarPorValor(dado) != dado:# valor nao esta no grafo
            self.__vertices.add(Vertice(dado))
            return True
        return False
    
    def adicionarAresta(self, origem, destino, peso = 1, direcionada = True):
        try:
            verticeOrigem = self.getArestaDado(origem)
            verticeDestino = self.getArestaDado(destino)
            if (verticeOrigem or verticeDestino) is None:  # existem os vertices de origem e destino?
                print("Nao ha no grafo, vertices de origem ou de destino com os valores informados.")
            self.__arestas.add(Aresta(verticeOrigem, verticeDestino, peso, direcionada))
        except AttributeError as error:
            print("Nao ha no grafo, vertices de origem ou de destino com os valores informados.")
    
    def removeVertice(self, dado): 
            if self.buscarPorValor(dado) != dado:
                print("O valor informado nao esta no grafo.")
            else:
                for a in self.getVertice():
                    if a.getDado == dado:
                        a.remove(Vertice(dado))
                return False
            
    def removeAresta(self, valorOrigem, valorDestino, peso):
        try:
            o = self.getArestaDado(valorOrigem)
            d = self.getArestaDado(valorDestino)
            for a in self.getArestas():
                if a.getDado == peso and a.getvOrigem() == o and a.getvDestino == d:
                    self.getArestas().remove(a)
        except AttributeError as error:
            print("Nao ha no grafo, Arestas de origem ou de destino com os valores informados.")
                    
#-------11/05-------#

    def HandShakingLemma(self):
        return sum(v.getGrau() for v in self.getVertices()) == 2*len(self.getArestas())
    
    '''  27/04
    def KL9(self, valor, visitados=[]):
        V = self.getVertices()
        if valor not in visitados:
            visitados.append(valor)
            for i in V:
                if i.getValor() not in visitados:
                    return visitados
                else:
                    for visitados in self.getVertices():
                        visitados = self.KL9(i.getValor(), visitados)
        return visitados
    '''
#-------04/05-------#
#- Busca em profundidade:
    def asd(self, graph, v, visit=[]):
        if v not in visit:
            visit.append(v)
        if len(v.getAdjacentes(v)) == 0:
            self.asd(graph, next(iter(self.getVertices())), visit)
        else:
            for adjacente in v.getAdjacentes(v):
                if adjacente not in visit:
                    self.asd(graph, adjacente, visit)
        return visit
    
#- Busca em largura:
    def qwe(self, v, visit = [], fila = deque([])):
        fila.append(v)
        if v not in visit:
            visit.append(v)
        while fila:
            vertice = fila.popleft()
            if len(vertice.getArestasSaida()) == 0:
                self.qwe(next(iter(self.getVertices())), visit, fila)
            else:
                for i in vertice.getArestasSaida():
                    vAdjacente = i.getvDestino()
                    if vAdjacente not in visit:
                        visit.append(vAdjacente)
                        fila.append(vAdjacente)
        return visit
    
    def buscarPorValor(self, dado):
        for v in self.qwe(next(iter(self.getVertices())), visit = [], fila = deque([])):
            if dado == v.getDado():
                return dado
        return None
    
#-------04/05-------#

In [ ]:
    '''
    def adicionarVertice(self, dado):
        if self.buscarPorValor(dado) != dado:
            self.__vertices.add(Vertice(dado))
            return True
        return False
    
    def adicionarAresta(self, origem, destino, peso = 1, direcionada = True):
        try:
            verticeOrigem = self.getVerticeByValor(origem)
            verticeDestino = self.getVerticeByValor(destino)
            if (verticeOrigem or verticeDestino) is None:  # existem os vertices de origem e destino?
                print("Nao ha no grafo, vertices de origem ou de destino com os valores informados.")
            self.__arestas.add(Aresta(verticeOrigem, verticeDestino, peso, direcionada))
        except AttributeError as error:
            print("Nao ha no grafo, vertices de origem ou de destino com os valores informados.")
    '''         
    def removeVertice(self, dado):
        if self.buscarPorValor(dado) != dado:
            print("O valor informado nao esta no grafo.")
        return vertices
        else:
            for a in self.getVertice():
                if a.getDado == dado:
                self.get.remove(Vertice(dado))            
        return vertices   

    def rVertice(self, dado, origem, destino):
        for b in self.buscarPorValor():
            if b.getDado(dado) == dado:
                self.Vertice().remove(dado)
                
    
    def removeAresta(self, valorOrigem, valorDestino, peso):
        try:
            o = self.getArestaDado(valorOrigem)
            d = self.getArestaDado(valorDestino)
            for a in self.getArestas():
                if a.getDado == peso and a.getvOrigem() = o and a.getvDestino == d:
                    self.getArestas().remove(a)
                    
#-------18/05-------#

In [6]:
#   def htr(self):
#        if 

In [7]:
    def getsnoe(self):
        if self.__getDado == vertices:
            return self.__vertices
        v.getVertices = []
        for vertices in self.__Vertices:
            if vertices.getDado() == True:
                verticesSaida.append(vertices)
                return vertices
            else:
                vertices.getDado() == False
                return vertices
        return v.getVertices

In [24]:
v1 = Vertice(1)
v1.getDado()

1

In [25]:
v2 = Vertice(2)
v2.getDado()

2

In [26]:
v3 = Vertice(3)
v3.getDado()

3

In [27]:
v4 = Vertice(4)
v4.getDado()

4

In [28]:
a1 = Aresta( v1, v2, 1, True )
a2 = Aresta( v2, v3, 1, True )
a3 = Aresta( v3, v4, 1, True )
a4 = Aresta( v4, v1, 1, True )

In [29]:
for a in v2.getArestas():
    print(str(a.getvOrigem().getDado())+" |---> "+str(a.getvDestino().getDado()))
for a in v4.getArestas():
    print(str(a.getvOrigem().getDado())+" |---> "+str(a.getvDestino().getDado()))

1 |---> 2
2 |---> 3
4 |---> 1
3 |---> 4


In [30]:
G = Grafo()
G.setVertices({v1, v2, v3, v4})
G.setArestas({a1, a2, a3, a4})
for v in G.getVertices():
    print(v.getDado(), end="\t")

3	1	2	4	

In [31]:
for a in G.getArestas():
    print(a.getvOrigem().getDado(), end="")
    print(" ---> ", end="")
    print(a.getvDestino().getDado())

3 ---> 4
4 ---> 1
1 ---> 2
2 ---> 3


In [14]:
v1.getGrau()
#G.getArestaDado(3).getGrauE())
#G.getArestaDado(4).getGrauS())

4

In [15]:
G.HandShakingLemma()

False

In [32]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True )
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5})
G.setArestas({a1, a2, a3, a4, a5})

In [33]:
G.adicionarVertice(20)
G.adicionarAresta(1, 20, True)
#G.removeVertice(20)
G.removeAresta(1, 20, True)

In [34]:
for v in G.getVertices():
    print(v.getDado(), end="\t")

2	20	5	3	1	4	

In [35]:
for a in G.getArestas():
    print(a.getvOrigem().getDado(), end="")
    print(" ---> ", end="")
    print(a.getvDestino().getDado(), end="\t")

2 ---> 3	4 ---> 5	1 ---> 2	1 ---> 20	4 ---> 1	3 ---> 4	

In [36]:
v4.getGrau()

3

In [245]:
#04/05- Busca em profundidade:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
v6 = Vertice(6)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True ) 
a6 = Aresta( v4, v6, 60, True ) 
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5,v6})
G.setArestas({a1, a2, a3, a4, a5, a6})
for vertice in G.getVertices():
    print(f"Busca em profundidade, iniciando com o vértice {vertice.getDado()}:")
    for v in G.asd(G, vertice, visit=[]):
        print(str(v.getDado())+"\t", end="")
    print("\n.................................................")  

Busca em profundidade, iniciando com o vértice 1:
1	2	3	4	6	5	
.................................................
Busca em profundidade, iniciando com o vértice 4:
4	1	2	3	6	5	
.................................................
Busca em profundidade, iniciando com o vértice 6:
6	1	2	3	4	5	
.................................................
Busca em profundidade, iniciando com o vértice 5:
5	1	2	3	4	6	
.................................................
Busca em profundidade, iniciando com o vértice 3:
3	4	1	2	6	5	
.................................................
Busca em profundidade, iniciando com o vértice 2:
2	3	4	1	6	5	
.................................................


In [249]:
#04/05- Busca em largura:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
v6 = Vertice(6)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True ) 
a6 = Aresta( v4, v6, 60, True ) 
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5,v6})
G.setArestas({a1, a2, a3, a4, a5, a6})
for vertice in G.getVertices():
    print(f"Busca em largura, iniciando com o vértice {vertice.getDado()}:")
    for v in  G.qwe(vertice, visit = [], fila = deque([])):
        print(str(v.getDado())+"\t", end="")
    print("\n............................................") 

Busca em largura, iniciando com o vértice 1:
1	2	3	4	5	6	
............................................
Busca em largura, iniciando com o vértice 6:
6	1	2	3	4	5	
............................................
Busca em largura, iniciando com o vértice 4:
4	1	5	6	2	3	
............................................
Busca em largura, iniciando com o vértice 3:
3	4	1	5	6	2	
............................................
Busca em largura, iniciando com o vértice 5:
5	1	2	3	4	6	
............................................
Busca em largura, iniciando com o vértice 2:
2	3	4	1	5	6	
............................................


In [250]:
#class- Aresta
#         getValor
def getValor(self):
    return self._getValor()  
#class- Grafo
#         getAresta(valor)
def getArestaValor(self):
    return len(self.getAresta(getValor))

In [15]:
getGrauEntrada
getGrauSaida

    Aresta
        getValor
    Grafo
        getAresta(valor)

In [18]:
def getVertices1(self):
    for v in self.getVertices():
        if v.getVertices() == vertices:
            return v
        return vertices

In [6]:
l1= [v1,v2,v3,v4]
l2= [v1,v2,v3]

In [25]:
def na(self):
    for i in getArestas:
        if len(i.getDado())==True:
            self.getDado(i)
        else:
            self.getVertices(len(G.getDado))
            return arestas
            
    #getVertices
    #getArestas    

In [ ]:
dado = int(input("Valor procurado: "))
if G.buscarPorValor(dado) == None:
    print(f"{dado} não encontrado no grafo")
else:
    print(f"{dado} encontrado no grafo")